In [66]:
########################
## Create training data
######################

import numpy as np
import scipy.misc as smp
import random

#Create input array of the form: [1, 1, 1, 0, 0, 0, 1] (here N=3)
                             #     \ /\ /\ /\ /\ /\
#Create output array of the form:  [1, 1, 1, 0, 0, 1] 


arr_size_half = 20
max_N = 12


def create_inp_outp_array():
    #max_N = 8
    N = random.randint(1, max_N)
    #print("N=", N)
    inp_array = np.zeros(2*arr_size_half)
    inp_array[0:N] = 1
    inp_array[2*N:2*arr_size_half]=1
    
    out_array = np.zeros(2*arr_size_half)
    out_array[0:N] = 1
    out_array[2*N-1:2*arr_size_half]=1
    
    return inp_array, out_array
 

def make_data_set(n_samples):
  mult_inp, mult_out = create_inp_outp_array()
  for i in range(n_samples - 1):
      single_inp, single_out = create_inp_outp_array()
      mult_inp = np.vstack([mult_inp, single_inp])
      mult_out = np.vstack((mult_out, single_out))
  return mult_inp, mult_out



train_input, train_output = make_data_set(1000)
max_N=20
test_input, test_output = make_data_set(1000)


#Print single one
inp_ex, out_ex = create_inp_outp_array()
print("inp_ex", inp_ex)
print("out_ex", out_ex)

#Print set data
print("train_input: number of examples: ", train_input.shape)
print("train_input: number of examples: ", train_input[:,0].size)
print("train_input: single example size: ", train_input[0].size)
print("train_input: dim of elements in example: ", train_input[0,0].size)



inp_ex [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
out_ex [1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
train_input: number of examples:  (1000, 40)
train_input: number of examples:  1000
train_input: single example size:  40
train_input: dim of elements in example:  1


In [70]:
##############################
## A^N B^N
##################################

import numpy as np
import scipy.misc as smp
import random
#Source code with the blog post at http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/
import numpy as np
import random
from random import shuffle
import tensorflow as tf
tf.reset_default_graph()

# from tensorflow.models.rnn import rnn_cell
# from tensorflow.models.rnn import rnn

NUM_EXAMPLES = 10000

# Create input data

print("test and training data loaded")


inp_size = train_input[0].size
out_size = train_output[0].size

data = tf.placeholder(tf.float32, [None, inp_size,1]) #Number of examples, number of input, dimension of each input                # change shape: get first array length from input_data and output_data
target = tf.placeholder(tf.float32, [None, out_size,1])                                                                              # change shape: get first array length from input_data and output_data
num_hidden = 1                                                                                                              # maybe change this?
#cell = tf.contrib.rnn.LSTMCell(num_hidden,state_is_tuple=True)
cell = tf.nn.rnn_cell.GRUCell(num_hidden)
val, _ = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
val = tf.transpose(val, [1, 0, 2])
#last = tf.gather(val, int(val.get_shape()[0]) - 1)
last = val #--> not only take last.. do it in eager execution to check everything
############new
weight = tf.Variable(tf.truncated_normal([num_hidden, 1]))                                           #--> num_hidden, 1] 
bias = tf.Variable(tf.constant(0.1, shape=[1]))


#Get output of all steps
#transformed_outputs = [tf.matmul(val_i, weight + bias) for val_i in val]
transformed_outputs = tf.map_fn(lambda val_i: tf.matmul(val_i, weight + bias), val)


'''
i = tf.constant(0)
while_condition = lambda i: tf.less(i, train_input[0].size)
def body(i):
    # do something here which you want to do in your loop
    # increment i
    tf.matmul(val[i], weight + bias)
    return [tf.add(i, 1)]
# do the loop:
r = tf.while_loop(while_condition, body, [i])
'''

transformed_outputs = tf.transpose(transformed_outputs, [1, 0, 2])
binary_single_transformed_outputs =  tf.nn.sigmoid(transformed_outputs)
round_binary_single_transformed_outputs = tf.round(binary_single_transformed_outputs)
cross_entropy = tf.reduce_sum( (target - binary_single_transformed_outputs)*(target - binary_single_transformed_outputs) )




###########
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)
mistakes = tf.not_equal(target,  tf.round(binary_single_transformed_outputs))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

batch_size = 40
no_of_batches = int(int(len(train_input)) / batch_size)
epoch = 70
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp.reshape(batch_size, inp_size,1), target: out.reshape(batch_size, out_size,1)})
    print("Epoch ",str(i) )
    incorrect = sess.run(error,{data: test_input.reshape(test_input[:,0].size, inp_size,1), target: test_output.reshape(test_output[:,0].size, inp_size,1)})
    print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))

#create single-test example
inp_ex, out_ex = create_inp_outp_array()
print("Test input: ", inp_ex)
print("Should output: ", out_ex)
print("Predicted output: ")
print(sess.run(round_binary_single_transformed_outputs,{data: inp_ex.reshape(1, inp_size,1)}))
sess.close()
 




test and training data loaded
Epoch  0
Epoch  1 error 23.5%
Epoch  1
Epoch  2 error 23.5%
Epoch  2
Epoch  3 error 23.5%
Epoch  3
Epoch  4 error 23.5%
Epoch  4
Epoch  5 error 23.5%
Epoch  5
Epoch  6 error 23.5%
Epoch  6
Epoch  7 error 23.5%
Epoch  7
Epoch  8 error 23.5%
Epoch  8
Epoch  9 error 23.5%
Epoch  9
Epoch 10 error 23.5%
Epoch  10
Epoch 11 error 23.5%
Epoch  11
Epoch 12 error 23.5%
Epoch  12
Epoch 13 error 23.5%
Epoch  13
Epoch 14 error 23.5%
Epoch  14
Epoch 15 error 23.5%
Epoch  15
Epoch 16 error 23.5%
Epoch  16
Epoch 17 error 23.5%
Epoch  17
Epoch 18 error 23.5%
Epoch  18
Epoch 19 error 23.5%
Epoch  19
Epoch 20 error 23.5%
Epoch  20
Epoch 21 error 23.5%
Epoch  21
Epoch 22 error 23.5%
Epoch  22
Epoch 23 error 23.5%
Epoch  23
Epoch 24 error 23.5%
Epoch  24
Epoch 25 error 23.5%
Epoch  25
Epoch 26 error 23.5%
Epoch  26
Epoch 27 error 19.4%
Epoch  27
Epoch 28 error 15.6%
Epoch  28
Epoch 29 error 12.6%
Epoch  29
Epoch 30 error 10.8%
Epoch  30
Epoch 31 error 10.8%
Epoch  31
Epoch 32 

In [24]:
##############################
## A^N B^N  - EAGER EXECUTION  for debugging
##################################

import numpy as np
import scipy.misc as smp
import random
#Source code with the blog post at http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/
import numpy as np
import random
from random import shuffle
import tensorflow as tf
tf.reset_default_graph()
tf.enable_eager_execution()

# from tensorflow.models.rnn import rnn_cell
# from tensorflow.models.rnn import rnn

NUM_EXAMPLES = 10000

# Create input data

print("test and training data loaded")


inp_size = train_input[0].size
out_size = train_output[0].size



#data = tf.placeholder(tf.float32, [None, inp_size,1]) #Number of examples, number of input, dimension of each input                # change shape: get first array length from input_data and output_data
#target = tf.placeholder(tf.float32, [None, out_size])                                                                              # change shape: get first array length from input_data and output_data
batch_size = 1000
ptr = 0
data = tf.convert_to_tensor(train_input[ptr:ptr+batch_size].reshape(batch_size, inp_size,1), dtype=tf.float32 )
target = tf.convert_to_tensor(train_output[ptr:ptr+batch_size].reshape(batch_size, out_size,1), dtype=tf.float32 )
num_hidden = 24                                                                                                              # maybe change this?
cell = tf.contrib.rnn.LSTMCell(num_hidden,state_is_tuple=True)
val, _ = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
val = tf.transpose(val, [1, 0, 2])




print("val.shape: ", tf.shape(val))
last = tf.gather(val, int(val.get_shape()[0]) - 1)
print("last.shape: ", tf.shape(last))
last = val #--> not only take last.. do it in eager execution to check everything
weight = tf.Variable(tf.truncated_normal([num_hidden, 1]))                                           #--> num_hidden, 1] 
bias = tf.Variable(tf.constant(0.1, shape=[1]))


#Get output of all steps
transformed_outputs = [tf.matmul(val_i, weight + bias) for val_i in val]
print("transformed_outputs.shape: ", tf.shape(transformed_outputs))
transformed_outputs = tf.transpose(transformed_outputs, [1, 0, 2])
print("transformed_outputs.shape: ", tf.shape(transformed_outputs))

single_transformed_outputs = transformed_outputs #tf.gather(transformed_outputs, 0)
print("single_transformed_outputs.shape: ", tf.shape(single_transformed_outputs))
binary_single_transformed_outputs =  tf.round(tf.nn.sigmoid(single_transformed_outputs))
print("binary_single_transformed_outputs: ", binary_single_transformed_outputs)


#prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
cross_entropy = -tf.reduce_sum(target * binary_single_transformed_outputs)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

batch_size = 1000
no_of_batches = int(int(len(train_input)) / batch_size)
epoch = 20
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    print("Epoch ",str(i) )
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
print(np.argmax(sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]})) )           #Change this to appropriate input
sess.close()
 




test and training data loaded
val.shape:  tf.Tensor([   8 1000   24], shape=(3,), dtype=int32)
last.shape:  tf.Tensor([1000   24], shape=(2,), dtype=int32)
transformed_outputs.shape:  tf.Tensor([   8 1000    1], shape=(3,), dtype=int32)
transformed_outputs.shape:  tf.Tensor([1000    8    1], shape=(3,), dtype=int32)
single_transformed_outputs.shape:  tf.Tensor([1000    8    1], shape=(3,), dtype=int32)
binary_single_transformed_outputs:  tf.Tensor(
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]], shape=(1000, 8, 1), dtype=float32)


RuntimeError: ignored